# Pubmed Crawler
Author: Brian Bacik

In [372]:
import pandas as pd
import datetime
from datetime import date
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlite3
import numpy as np
import boto3

# Module 1

Pubmed crawler module tht can collect paper title, author list, publication time, and abstract from PUBMED for a given keyword (i.e., COVID-19 Vaccine) within a specified time window.

In [381]:
#get date ranges to use in pubmed query
today = date.today()
cur_date = today.strftime("%Y/%m/%d")

last_week = (today-datetime.timedelta(days=7)).strftime("%Y/%m/%d")
print(cur_date, last_week)

2021/03/08 2021/03/01


### Pubmed crawler function

In [173]:
#write Pubmed search query
from Bio import Entrez

#returns list of article ids that contain keyword 
def search(query):
    Entrez.email = 'bdbacik@gmail.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='100000',
                            retmode='xml',
                            datetype='pdat',
                            mindate=last_week,
                            maxdate=cur_date,
                            term=query)
    search_results = Entrez.read(handle)
    return search_results

#users article ids as input and returns dictionary with article details
def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'bdbacik@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    fetch_results = Entrez.read(handle)
    return fetch_results


if __name__ == '__main__':
    search_results = search('COVID-19 vaccine')
    id_list = search_results['IdList']
    fetch_results = fetch_details(id_list)

In [382]:
print('length of esearch output: ', len([item for item in search_results['IdList']]))
print('length of efetch output: ', len([key for key in fetch_results['PubmedArticle']]))

length of esearch output:  4335
length of efetch output:  4333


### Convert crawler output to pandas dataframe

In [317]:
#pull relevant fields from query into pandas dataframe

searchoutput = {"Title":[], "Keywords":[], "PublicationDate": [], "Authors": [], 
                "Abstract": [], 'Country':[]}
for i, paper in enumerate(fetch_results['PubmedArticle']): 
    try:
        Title = paper['MedlineCitation']['Article']['ArticleTitle']
        Keywords = paper['MedlineCitation']['KeywordList']
        PublicationDate = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']#['Month']
        Authors = paper['MedlineCitation']['Article']['AuthorList']
        Abstract = paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
        Country = paper['MedlineCitation']['MedlineJournalInfo']['Country']
    except KeyError as e:
        continue
    searchoutput["Title"].append(Title)
    searchoutput["Keywords"].append(Keywords)
    searchoutput["PublicationDate"].append(PublicationDate)
    searchoutput["Authors"].append(Authors)
    searchoutput["Abstract"].append(str(Abstract))
    searchoutput["Country"].append(Country)

df = pd.DataFrame(searchoutput)

df.head()

,Title,Keywords,PublicationDate,Authors,Abstract,Country
0,COVID-19 vaccines: ethical framework concernin...,"[[COVID-19 vaccines, Human challenge studies, ...","{'Year': '2020', 'Month': 'Dec'}","[{'AffiliationInfo': [{'Identifier': [], 'Affi...",The pandemic associated with the new SARS-CoV-...,Switzerland
1,Coronavirus Disease 2019 (COVID-19) Vaccines a...,[],"{'Year': '2021', 'Month': '03', 'Day': '01'}","[{'AffiliationInfo': [{'Identifier': [], 'Affi...",Coronavirus disease 2019 (COVID-19) vaccines h...,United States
2,Unfolding the Determinants of COVID-19 Vaccine...,"[[COVID-19, COVID-19 vaccination, COVID-19 vac...","{'Year': '2021', 'Month': '01', 'Day': '15'}","[{'AffiliationInfo': [{'Identifier': [], 'Affi...",China is at the forefront of global efforts to...,Canada
3,Learning from the past: development of safe an...,[],"{'Year': '2021', 'Month': '03'}","[{'AffiliationInfo': [{'Identifier': [], 'Affi...",The rapid spread of severe acute respiratory s...,England
4,COVID-19 Vaccines: Preparing for Vaccination i...,"[[COVID-19 vaccine, cancer, cancer program saf...","{'Year': '2021', 'Month': 'Feb', 'Day': '01'}","[{'AffiliationInfo': [{'Identifier': [], 'Affi...","In the environment of an infectious pandemic, ...",United States


In [319]:
#get publication date in YYYYMM 
df['Pub_Year'] = [str(d.get('Year')) for d in df['PublicationDate']]
df['Pub_Month'] = [str(d.get('Month')) for d in df['PublicationDate']]
df["Pub_Month"] = df["Pub_Month"].replace(["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],["01","02","03","04","05","06","07","08","09","10","11","12"])
df['Pub_Date'] = pd.to_datetime(df.Pub_Year+'-'+df.Pub_Month, errors='ignore')
df = df.drop(columns=['PublicationDate','Pub_Year','Pub_Month'])
df.head()

,Title,Keywords,Authors,Abstract,Country,Pub_Date
0,COVID-19 vaccines: ethical framework concernin...,"[[COVID-19 vaccines, Human challenge studies, ...","[{'AffiliationInfo': [{'Identifier': [], 'Affi...",The pandemic associated with the new SARS-CoV-...,Switzerland,2020-12
1,Coronavirus Disease 2019 (COVID-19) Vaccines a...,[],"[{'AffiliationInfo': [{'Identifier': [], 'Affi...",Coronavirus disease 2019 (COVID-19) vaccines h...,United States,2021-03
2,Unfolding the Determinants of COVID-19 Vaccine...,"[[COVID-19, COVID-19 vaccination, COVID-19 vac...","[{'AffiliationInfo': [{'Identifier': [], 'Affi...",China is at the forefront of global efforts to...,Canada,2021-01
3,Learning from the past: development of safe an...,[],"[{'AffiliationInfo': [{'Identifier': [], 'Affi...",The rapid spread of severe acute respiratory s...,England,2021-03
4,COVID-19 Vaccines: Preparing for Vaccination i...,"[[COVID-19 vaccine, cancer, cancer program saf...","[{'AffiliationInfo': [{'Identifier': [], 'Affi...","In the environment of an infectious pandemic, ...",United States,2021-02


In [320]:
df['Pub_Date'].value_counts()

2021-02      458
2021-01      389
2020-12      367
2020-11      295
2020-10      287
2020-None    273
2020-09      242
2020-08      213
2020-07      192
2020-06      152
2021-03      113
2020-05      106
None-None     96
2021-None     58
2020-04       55
2020-03       27
2021-04       24
2021-05        5
2020-02        1
2021-12        1
2021-06        1
Name: Pub_Date, dtype: int64

### Create separate dataframes for multi-value attribute fields

In [321]:
#make separate df for multi value attribute column - keywords
keywords = df.explode('Keywords').explode('Keywords').iloc[:,:2]
keywords['Keywords'] = keywords['Keywords'].str.lower()
keywords.head(10)

,Title,Keywords
0,COVID-19 vaccines: ethical framework concernin...,covid-19 vaccines
0,COVID-19 vaccines: ethical framework concernin...,human challenge studies
0,COVID-19 vaccines: ethical framework concernin...,randomized clinical trials
0,COVID-19 vaccines: ethical framework concernin...,risk taking
1,Coronavirus Disease 2019 (COVID-19) Vaccines a...,NaN
2,Unfolding the Determinants of COVID-19 Vaccine...,covid-19
2,Unfolding the Determinants of COVID-19 Vaccine...,covid-19 vaccination
2,Unfolding the Determinants of COVID-19 Vaccine...,covid-19 vaccines
2,Unfolding the Determinants of COVID-19 Vaccine...,china
2,Unfolding the Determinants of COVID-19 Vaccine...,acceptance


In [322]:
#create new dataframe with columns for first and last name of each author of each paper
authors = df.explode('Authors')['Authors'].apply(pd.Series).loc[:,['LastName','ForeName','AffiliationInfo']]
authors["Author"] = authors["ForeName"].str.cat(authors["LastName"], sep=" ")
#merge original df and df2 with author names
df3 = pd.merge(df,authors,left_index=True,right_index=True)
df3 = df3[["Title", "Author", 'AffiliationInfo']]
df3['AffiliationInfo'] = df3['AffiliationInfo'].astype(str)
df3['AffiliationInfo'] = df3.apply(lambda x: x['AffiliationInfo'].split('Affiliation')[-1], axis=1).str[3:-2]
df3.head()

,Title,Author,AffiliationInfo
0,COVID-19 vaccines: ethical framework concernin...,Daniela Calina,"'Department of Clinical Pharmacy, University o..."
0,COVID-19 vaccines: ethical framework concernin...,Thomas Hartung,"'CAAT, Bloomberg School of Public Health, John..."
0,COVID-19 vaccines: ethical framework concernin...,Anca Oana Docea,"'Department of Toxicology, University of Medic..."
0,COVID-19 vaccines: ethical framework concernin...,Demetrios A Spandidos,"'Laboratory of Clinical Virology, Medical Scho..."
0,COVID-19 vaccines: ethical framework concernin...,Alex M Egorov,"'Russian Academy of Sciences, Moscow, Russia.'"


In [323]:
df = df.drop(columns=['Keywords','Authors'])
df.head()

,Title,Abstract,Country,Pub_Date
0,COVID-19 vaccines: ethical framework concernin...,The pandemic associated with the new SARS-CoV-...,Switzerland,2020-12
1,Coronavirus Disease 2019 (COVID-19) Vaccines a...,Coronavirus disease 2019 (COVID-19) vaccines h...,United States,2021-03
2,Unfolding the Determinants of COVID-19 Vaccine...,China is at the forefront of global efforts to...,Canada,2021-01
3,Learning from the past: development of safe an...,The rapid spread of severe acute respiratory s...,England,2021-03
4,COVID-19 Vaccines: Preparing for Vaccination i...,"In the environment of an infectious pandemic, ...",United States,2021-02


## Module 2 - Database Development
1. Builds SQLite database to store our pubmed crawler output. 
2. Create SQL tables to represent data from module 1.
3. Query tables to get key statistics and setup for visualization.

In [324]:
%cd /Users/Brian/Dropbox/Data_Science/Github portfolio/Pubmed crawler/

/Users/Brian/Dropbox/Data_Science/Github portfolio/Pubmed crawler


### Create database

In [325]:
conn = sqlite3.connect('pubmed.db')  # create a new database or connect to database if already exists
c = conn.cursor() # create connection object

### Create tables and add data to them
We have three tables in the database:
1. pub_info - this contains the main information about the publication, including author, abstract, country published in, and publication date
2. authors - this table contains author name and affiliaition, with title as foreign key
3. keywords - this table contains keywords for each article, with title as foreign key

In [326]:
# Create pub_info table
try:
  c.execute('''CREATE TABLE pub_info
              ([Title] text, [Abstract] text, [Country] text, [Pub_Date] text)''')
  conn.commit()
  print('SQL table created successfully!')

except:
  print('SQL table already exists!')

SQL table already exists!


In [327]:
#add data to sql table from our pandas dataframe
df.to_sql('pub_info', conn, if_exists='append', index=False)

In [328]:
#query database to get the number of publications by country
pubs_by_country = c.execute('''SELECT DISTINCT Country, count() OVER(PARTITION BY Country) as Num_Publications
                            FROM pub_info 
                            ORDER BY Num_Publications DESC 
                            ''').fetchall()
pubs_by_country[0:10]

[('United States', 1157),
 ('England', 720),
 ('Switzerland', 481),
 ('Netherlands', 352),
 ('Germany', 107),
 ('China', 53),
 ('Canada', 47),
 ('Italy', 42),
 ('France', 38),
 ('India', 33)]

In [329]:
#query database to get the number of publications by month
pubs_by_month = c.execute('''SELECT Pub_date, count(Pub_Date) as Pubs_by_month
                            FROM pub_info 
                            GROUP BY Pub_date
                            ORDER BY Pubs_by_month DESC 
                            ''').fetchall()
pubs_by_month

[('2021-02', 458),
 ('2021-01', 389),
 ('2020-12', 367),
 ('2020-11', 295),
 ('2020-10', 287),
 ('2020-None', 273),
 ('2020-09', 242),
 ('2020-08', 213),
 ('2020-07', 192),
 ('2020-06', 152),
 ('2021-03', 113),
 ('2020-05', 106),
 ('None-None', 96),
 ('2021-None', 58),
 ('2020-04', 55),
 ('2020-03', 27),
 ('2021-04', 24),
 ('2021-05', 5),
 ('2021-12', 1),
 ('2021-06', 1),
 ('2020-02', 1)]

In [330]:
# Create authors table
try:
  c.execute('''CREATE TABLE authors
              ([Title] text, [Author] text, [AffiliationInfo] text)''')
  conn.commit()
  print('SQL table created successfully!')

except:
  print('SQL table already exists!')

SQL table already exists!


In [331]:
#add data to sql table from our pandas dataframe
df3.to_sql('authors', conn, if_exists='append', index=False)

In [332]:
#query database to find top 10 authors by number of publications
top_authors = c.execute('''SELECT DISTINCT Author, count(Author) as Count, AffiliationInfo
                        FROM authors 
                        WHERE Author IS NOT NULL
                        GROUP BY Author
                        ORDER BY Count DESC 
                        LIMIT 10''').fetchall()
top_authors[0:5]

[('Ralph S Baric',
  25,
  "'Department of Epidemiology, Gillings School of Public Health, University of North Carolina at Chapel Hill, Chapel Hill, NC, USA.'"),
 ('Kuldeep Dhama',
  25,
  "'Division of Pathology, ICAR-Indian Veterinary Research Institute , Bareilly, India.'"),
 ('Florian Krammer',
  22,
  "'Department of Microbiology, Icahn School of Medicine at Mount Sinai, New York, NY 10029, United States.'"),
 ('Pei-Yong Shi',
  17,
  '"From the University of Rochester and Rochester General Hospital, Rochester (E.E.W., A.R.F.), Vaccine Research and Development, Pfizer, Pearl River (J.A., A.G., K.A.S., K.K., W.K., D.C., K.R.T., P.R.D., K.U.J., W.C.G.), and New York University Langone Vaccine Center and Grossman School of Medicine, New York (M.J.M., V.R.) - all in New York; Cincinnati Children\'s Hospital, Cincinnati (R.W.F.); Vaccine Research and Development, Pfizer, Hurley, United Kingdom (N.K., S.L., R.B.); the University of Maryland School of Medicine, Center for Vaccine Develop

In [333]:
# Create keywords table
try:
  c.execute('''CREATE TABLE keywords
              ([Title] text, [Keywords] text)''')
  conn.commit()
  print('SQL table created successfully!')

except:
  print('SQL table already exists!')

SQL table already exists!


In [334]:
#add data to sql table from our pandas dataframe
keywords.to_sql('keywords', conn, if_exists='append', index=False)

In [335]:
#query database to find top 10 keywords
top_keywords = c.execute('''SELECT Keywords, count(Keywords) as Count
                        FROM keywords
                        GROUP BY Keywords
                        ORDER BY Count DESC 
                        LIMIT 20''').fetchall()
top_keywords[0:20]

[('covid-19', 1634),
 ('sars-cov-2', 1094),
 ('coronavirus', 407),
 ('vaccine', 327),
 ('vaccines', 224),
 ('pandemic', 170),
 ('vaccination', 106),
 ('epidemiology', 72),
 ('spike protein', 63),
 ('clinical trials', 63),
 ('treatment', 53),
 ('public health', 53),
 ('vaccine hesitancy', 51),
 ('severe acute respiratory syndrome coronavirus 2', 49),
 ('sars', 49),
 ('ace2', 49),
 ('2019-ncov', 45),
 ('influenza', 44),
 ('immunization', 43),
 ('coronavirus disease 2019', 43)]

## Module 3 - Visualization
Use SQL query output to produce a dashboard summarizing key statistics related to publications

### 1. Publications by month

In [343]:
#convert SQL query output to pandas df for visualization
df_pubs_by_month = pd.DataFrame(pubs_by_month, columns=['Pub_Date', 'Count'])
df_pubs_by_month['Pub_Date'] = df_pubs_by_month['Pub_Date'].astype(str)
df_pubs_by_month.head()

,Pub_Date,Count
0,2021-02,458
1,2021-01,389
2,2020-12,367
3,2020-11,295
4,2020-10,287


### 2. Publications by country

In [338]:
#convert SQL query output to pandas df for visualization
df_pubs_by_country = pd.DataFrame(pubs_by_country, columns=['Country', 'Num_Publications'])
total_pubs = df_pubs_by_country['Num_Publications'].sum()
print(total_pubs)
df_pubs_by_country['Percent_Pubs'] = df_pubs_by_country['Num_Publications']/total_pubs
df_pubs_by_country.head()

3355


,Country,Num_Publications,Percent_Pubs
0,United States,1157,0.344858
1,England,720,0.214605
2,Switzerland,481,0.143368
3,Netherlands,352,0.104918
4,Germany,107,0.031893


### 3. Publications by author

In [384]:
df_top_authors = pd.DataFrame(top_authors,columns=['Author', 'Count', 'Affiliation'])
df_top_authors.head()

,Author,Count,Affiliation
0,Ralph S Baric,25,"'Department of Epidemiology, Gillings School o..."
1,Kuldeep Dhama,25,"'Division of Pathology, ICAR-Indian Veterinary..."
2,Florian Krammer,22,"'Department of Microbiology, Icahn School of M..."
3,Pei-Yong Shi,17,"""From the University of Rochester and Rocheste..."
4,Ruchi Tiwari,16,'Department of Veterinary Microbiology and Imm...


### Create Dashboard
using plotly

In [385]:
#create dashboard to visualize publications by month, trend over time, and summary statistics

#define subplot figure contents
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Publications by Month", "Publications by Country", "Top Authors by Number of Publications"),
    shared_xaxes=False,
    vertical_spacing=0.1,
    specs=[[{"type": "bar"}],
           [{"type": "bar"}],
           [{"type": "table"}]])

#create histogram in first row
fig.add_trace(go.Bar(x=df_pubs_by_month.loc[:12,'Pub_Date'], y=df_pubs_by_month.loc[:12,'Count'],text=df_pubs_by_month.loc[:12,'Count'],
                     textposition='auto',name='Publications by Month'), row=1, col=1 )
fig.update_xaxes(
    dtick=1,
    tick0=1,
    tickformat="%m\n%Y", type='category')
#fig.update_layout(xaxis = dict(tickformat="%b\n%Y" ))

#create boxplot in second row
fig.add_trace(go.Bar(x=df_pubs_by_country.loc[:10,'Country'], y=df_pubs_by_country.loc[:10,'Num_Publications'],text=df_pubs_by_country.loc[:10,'Num_Publications'],
                     textposition='auto',name='Publications by Country'), row=2, col=1 )
fig.update_layout(xaxis = dict(tickmode = 'linear',tick0 = 1,dtick = 1 ))

#create summary statistics table in thirs row
fig.add_trace(go.Table(header=dict(values=['Author', 'Publications', 'Affiliations']), 
                       cells=dict(values=[df_top_authors.loc[:2,'Author'], df_top_authors.loc[:2,'Count'], 
                                          df_top_authors.loc[:2,'Affiliation']])), row=3, col=1)

#define layout
fig.update_layout(
    height=1000,
    showlegend=False,
    title_text="PubMed Publications for Research Related to Covid-19 Vaccine")

fig.show()

### Save dashboard image to local machine then upload to AWS

In [370]:
#save image to local machine
fig.write_image("pubmed_dashboard2.png", scale=1, width=1200, height=1200)

In [374]:
#upload image to AWS
s3 = boto3.resource('s3')
s3.meta.client.upload_file('pubmed_dashboard2.png', 'pubmedcrawler', 'pubmed_dashboard2.png', 
                           ExtraArgs={'ACL':'public-read'})